ある程度上位には入れて、これ以上はもう良いだろうという気持ちになったので方針を書きます

* 提出はパラメーターを少し変えて5つしてます
* agent1(初日, 650くらい)
  * 終了するタイミングがわかっている多腕バンディットは、UCBよりSoftmax Action Selection(?名称がよくわかってない)の方がよかった記憶があるので、単純に実装
  * 自分の行動を記録しておけば、相手の行動は　sum(lastActions) - 自分の行動 でわかる
    * バージョンアップして必要なくなったっぽいが
  * hits / cnts で減衰していない値を得たかったので、hitsに+するときに割ってる
* agent2(二日目, 900弱?)
  * 相手の行動を考慮するようにした
  * 相手が認識している優先順位を回数と順序で推定、優先順位順に99-0を割り当てる
  * 不確実な相手の結果より自分の結果の方を優先したいので重み付けを入れた
* agent3(三日目, 9連勝して1000を超えてるのもある)
  * パラメーター調整が不十分だったことがわかったので調整した

In [ ]:
!pip install kaggle-environments --upgrade -q

In [ ]:
%%writefile agent1.py


import math, random


history = {
    "turn": 0,
    "cnts": [0] * 100,
    "ocnts": [0] * 100,
    "hits": [0] * 100,
    "la": -1,
}

def agent1(observation, configuration):
    global history

    N = 100
    p = [63.9413, 0.000533149]
    ti = observation["step"]
    if ti == 0:
        pass
    else:
        la = history["la"]
        ola = sum(observation['lastActions']) - la
        if sum(history["hits"]) < observation['reward']:
            history["hits"][la] += 1 / pow(0.97, history["cnts"][la] + history["ocnts"][la])
        history["cnts"][la] += 1
        history["ocnts"][ola] += 1

    tau = p[0] / (ti + 1) + p[1]
    ea = [0] * N
    hits = history["hits"]
    cnts = history["cnts"]
    ocnts = history["ocnts"]
    for i in range(N):
        if cnts[i] == 0:
            ea[i] = math.exp(0.99 * pow(0.97, ocnts[i]) / tau)
        else:
            ea[i] = math.exp(hits[i] / cnts[i] * pow(0.97, cnts[i] + ocnts[i]) / tau)

    se = sum(ea)
    r = random.random() * se
    t = 0
    la = 99
    for i in range(N):
        t += ea[i]
        if t >= r:
            la = i
            break

    history["la"] = la
    return la


In [ ]:
%%writefile agent2.py

import math, random


history = {
    "turn": 0,
    "cnts": [0] * 100,
    "ocnts": [0] * 100,
    "hits": [0] * 100,
    "osteps": [0] * 100,
    "la": -1,
}

def agent2(observation, configuration):
    global history

    N = 100
    p = [4.60575, 0.000629018, 1.82229]
    ti = observation["step"]
    if ti == 0:
        pass
    else:
        la = history["la"]
        ola = sum(observation['lastActions']) - la
        history["osteps"][ola] = ti
        if sum(history["hits"]) < observation['reward']:
            history["hits"][la] += 1 / pow(0.97, history["cnts"][la] + history["ocnts"][la])
        history["cnts"][la] += 1
        history["ocnts"][ola] += 1

    tau = p[0] / (ti + 1) + p[1]
    ea = [0] * N
    hits = history["hits"]
    cnts = history["cnts"]
    ocnts = history["ocnts"]
    osteps = history["osteps"]

    tv = sorted([(-ocnts[i], osteps[i], i) for i in range(N)])
    ot = [0] * N
    for i in range(N):
        ot[tv[i][2]] = 99 - i

    for i in range(N):
        if cnts[i] == 0:
            if ocnts[i] > 1:
                ea[i] = math.exp(ot[i] / 100 * pow(0.97, ocnts[i]) / tau)
            else:
                ea[i] = math.exp(0.99 * pow(0.97, ocnts[i]) / tau)
        else:
            w = pow(cnts[i], p[2])
            wo = ocnts[i]
            if ocnts[i] < 2:
                wo = 0
            r = hits[i] / cnts[i]
            ro = ot[i] / 100
            ea[i] = math.exp((r * w + ro * wo) / (w + wo) * pow(0.97, cnts[i] + ocnts[i]) / tau)

    se = sum(ea)
    r = random.random() * se
    t = 0
    la = 99
    for i in range(N):
        t += ea[i]
        if t >= r:
            la = i
            break

    history["la"] = la
    return la


In [ ]:
%%writefile agent3.py

import math, random


history = {
    "turn": 0,
    "cnts": [0] * 100,
    "ocnts": [0] * 100,
    "hits": [0] * 100,
    "osteps": [0] * 100,
    "la": -1,
}

def agent3(observation, configuration):
    global history

    N = 100
    p = [0.39918, 0.000138129, 1.23946]
    ti = observation["step"]
    if ti == 0:
        pass
    else:
        la = history["la"]
        ola = sum(observation['lastActions']) - la
        history["osteps"][ola] = ti
        if sum(history["hits"]) < observation['reward']:
            history["hits"][la] += 1 / pow(0.97, history["cnts"][la] + history["ocnts"][la])
        history["cnts"][la] += 1
        history["ocnts"][ola] += 1

    tau = p[0] / (ti + 1) + p[1]
    ea = [0] * N
    hits = history["hits"]
    cnts = history["cnts"]
    ocnts = history["ocnts"]
    osteps = history["osteps"]

    tv = sorted([(-ocnts[i], osteps[i], i) for i in range(N)])
    ot = [0] * N
    for i in range(N):
        ot[tv[i][2]] = 99 - i

    for i in range(N):
        if cnts[i] == 0:
            if ocnts[i] > 1:
                ea[i] = math.exp(min(500, ot[i] / 100 * pow(0.97, ocnts[i]) / tau))
            else:
                ea[i] = math.exp(min(500, 0.99 * pow(0.97, ocnts[i]) / tau))
        else:
            w = pow(cnts[i], p[2])
            wo = ocnts[i]
            if ocnts[i] < 2:
                wo = 0
            r = hits[i] / cnts[i]
            ro = ot[i] / 100
            ea[i] = math.exp(min(500, (r * w + ro * wo) / (w + wo) * pow(0.97, cnts[i] + ocnts[i]) / tau))

    se = sum(ea)
    r = random.random() * se
    t = 0
    la = 99
    for i in range(N):
        t += ea[i]
        if t >= r:
            la = i
            break

    history["la"] = la
    return la


In [ ]:
from kaggle_environments import make, evaluate

agents = ["random", "agent1.py", "agent2.py", "agent3.py"]
run_count = 100

for i in range(4):
    for j in range(i+1, 4):
        ak = [0] * 2
        aw = [0] * 2

        for _ in range(run_count):
            env = make("mab")

            steps = env.run([agents[i], agents[j]])

            k = [0, 0]
            for s in steps[1:]:
                la = s[0]['observation']['lastActions']
                t = s[0]['observation']['thresholds']
                try:
                    k[0] += t[la[0]] / 97
                    k[1] += t[la[1]] / 97
                except Exception as e:
                    print(e)
                    print(s)

            ak[0] += k[0]
            ak[1] += k[1]
            if k[0] >= k[1]:
                aw[0] += 1
            else:
                aw[1] += 1

        print(f"{agents[i]} win: {aw[0]} avg: {ak[0] / run_count} vs {agents[j]} win: {aw[1]} avg: {ak[1] / run_count}")


その他

* パラメーター調整はC++でシュミレーションすると速い
* 評価はrewardより確率の合計の方が正確(なはず)
* このコンテストでより上位に入るためにこれからすべきこと
  * 毎日5つ提出し続ける
  * すべての提出物を評価対象にしてる理由がわからん